In [29]:
import pandas as pd
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# Load the data
df = pd.read_csv('data/get_around_pricing_project.csv', index_col=0)

print(df.head())

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4843 entries, 0 to 4842
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   model_key                  4843 non-null   object
 1   mileage                    4843 non-null   int64 
 2   engine_power               4843 non-null   int64 
 3   fuel                       4843 non-null   object
 4   paint_color                4843 non-null   object
 5   car_type                   4843 non-null   object
 6   private_parking_available  4843 non-null   bool  
 7   has_gps                    4843 non-null   bool  
 8   has_air_conditioning       4843 non-null   bool  
 9   automatic_car              4843 non-null   bool  
 10  has_getaround_connect      4843 non-null   bool  
 11  has_speed_regulator        4843 non-null   bool  
 12  winter_tires               4843 non-null   bool  
 13  rental_price_per_day       4843 non-null   int64 
dtypes: bool(7), i

In [ ]:
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Percentage of missing values: 


model_key                    0.0
mileage                      0.0
engine_power                 0.0
fuel                         0.0
paint_color                  0.0
car_type                     0.0
private_parking_available    0.0
has_gps                      0.0
has_air_conditioning         0.0
automatic_car                0.0
has_getaround_connect        0.0
has_speed_regulator          0.0
winter_tires                 0.0
rental_price_per_day         0.0
dtype: float64

In [ ]:
# Extract the features
X = df.drop('rental_price_per_day', axis=1)

# Extract the target column
y = df.loc[:, 'rental_price_per_day']

# Train / test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = 0.2)

In [ ]:
# determine categorical and numerical features
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'bool']).columns

# Numerical Transformer
numerical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
])

# Categorical Transformer
categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical_transformer", numerical_transformer, numerical_features),
        ("categorical_transformer", categorical_transformer, categorical_features)
    ]
)

# X_train = preprocessor.fit_transform(X_train)
# X_test = preprocessor.transform(X_test)

In [ ]:
# List of models
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    # ElasticNet(),
    # SVR(),
    # DecisionTreeRegressor(),
    # RandomForestRegressor(),
    # GradientBoostingRegressor(),
    # XGBRegressor(),
    # LGBMRegressor(),
    # CatBoostRegressor(verbose=0)
]

# List of param_grids for each model
param_grids = [
    {},  # Linear Regression
    {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, # Ridge
    {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}, # Lasso
    # {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}, # ElasticNet
    # {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [0.1, 1, 10, 100], 'epsilon': [0.01, 0.1, 1, 10]}, # SVR
    # {'max_depth': [2, 5, 10, 20], 'min_samples_split': [2, 5, 10]}, # Decision Tree Regressor
    # {'n_estimators': [100, 200, 500], 'max_depth': [2, 5, 10, 20], 'min_samples_split': [2, 5, 10]}, # Random Forest Regressor
    # {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [2, 5, 10], 'min_samples_split': [2, 5, 10]}, # Gradient Boosting Regressor
    # {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [2, 5, 10], 'min_child_weight': [1, 5, 10],'gamma': [0, 0.5, 1]}, # XGB Regressor
    # {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [2, 5, 10], 'num_leaves': [31]}, # LGBM Regressor
    # {'model__iterations': [100, 200, 500], 'model__learning_rate': [0.01, 0.1, 0.2], 'model__depth': [3, 6, 10], 'model__l2_leaf_reg': [1, 3, 5]}, # CatBoost Regressor
]

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Model', 'Best_Params', 'Best_Score'])

results = []

for i, model in enumerate(models):
    param_grid = param_grids[i]
    
    # Create a pipeline with the preprocessor and the model
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # Perform grid search with the current model and its param_grid
    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)

    # Store the best estimator and its score
    best_estimator = grid.best_estimator_
    best_score = grid.best_score_
    
    results.append((best_estimator, best_score))
    print(f"Best parameters for {model.__class__.__name__}: {grid.best_params_}")
    
# Print the results DataFrame
print(results_df)

NameError: name 'Ridge' is not defined